<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Introduction to Boosting

_Authors: Kiefer Katovich (SF), Matt Brems (DC)_


### Learning Objectives

- Understand the differences between bagging and boosting.

`Bagging is essentially bootstrapping aggregating. We take a random resampling with replacement of data of same size and create models, aggregating it to give a better prediction`

`Boosting helps to train the model based on misclassified samples by adding weights to the weaker base learners` 

- Understand how boosting is an ensemble method.

`**Base model fitting is an iterative procedure**: It cannot be run in parallel.`

`**Weights are assigned to observations to indicate their "importance:"** Samples with higher weights are given higher influence on the total error of the next model, prioritizing those observations.`

`**Weights change at each iteration with the goal of correcting the errors/misclassifications of the previous iteration**: The first base estimator is fit with uniform weights on the observations.`

`**Final prediction is typically constructed by a weighted vote**: Weights for each base model depend on their training errors or misclassification rates.`

- Learn the pros and cons to using boosting models.

`Higher performance but very time consuming to tune hyperparameters` 

- Learn the effect of boosting on the bias-variance trade-off.

`To reduce bias as essentially weak learners have low variance and high bias`

- Learn the math and procedure for AdaBoost, the "classic" boosting model.

- Understand the differences between AdaBoost and gradient-boosting models.

`AdaBoost is about reweighting the preceding model's errors in subsequent iterations.`
`Gradient boosting is about fitting subsequent models to the residuals of the last model.`


### Lesson Guide

- [Boosting as an Ensemble Method](#intro)
- [Pros and Cons of Boosting](#pros-cons)
- [A Visual Description of Bagging vs. Boosting](#viz)
- [Boosting and the Bias-Variance Trade-Off](#bias-variance)
- [AdaBoost](#adaboost)
    - [Training Example: Weights](#ex-weights)
- [AdaBoost Visualization](#adaboost-viz)
- [Gradient Boosting Models](#gradient)
- [Gradient Boosting Visualization](#gboost-viz)
- [The difference Between AdaBoost and Gradient Boosting](#the-difference-between-the-adaboost-and-gradient-boosting)
- [Additional Resources](#resources)

<a id='intro'></a>

## Boosting as an Ensemble Method

---

Boosting is another ensemble method with a different approach to bagging. Boosting takes a weak base learner and tries to make it a strong learner by retraining it on the misclassified samples.

1) **Base model fitting is an iterative procedure**: It cannot be run in parallel.
- **Weights are assigned to observations to indicate their "importance:"** Samples with higher weights are given higher influence on the total error of the next model, prioritizing those observations.
- **Weights change at each iteration with the goal of correcting the errors/misclassifications of the previous iteration**: The first base estimator is fit with uniform weights on the observations.
- **Final prediction is typically constructed by a weighted vote**: Weights for each base model depend on their training errors or misclassification rates.

<a id='pros-cons'></a>
## Pros and Cons of Boosting

---

### Pros

- Achieves higher performance than bagging when the hyperparameters are properly tuned.
- Works equally well for classification and regression.
- Can use "robust" loss functions that make the model resistant to outliers.

---

### Cons

- Difficult and time consuming to properly tune hyperparameters.
- Cannot be parallelized like bagging (bad scalability when there are huge amounts of data).
- Higher risk of overfitting compared to bagging.


<a id='viz'></a>
![boostvsbag](./images/BoostingVSBagging.png) 

<a id='bias-variance'></a>
## Boosting and the Bias-Variance Trade-Off

---

Recall that **bagging aims to reduce variance**.

**Boosting aims to reduce bias** (and can reduce variance a bit as well)!

### Why?

The rationale/theory behind boosting is to combine **many weak learners into a single strong learner.**

Instead of using deep/full decision trees like in bagging, **boosting uses shallow/high-bias base estimators.**

Thus, each weak learner has:

- Low variance.
- High bias.

It uses iterative fitting to explain error/misclassification unexplained by the previous base models and reduces bias without increasing variance.


# NOTES

Our goal is to build B trees 

at each stage we use full data 

we begin by just weighing each row equally
then we fit a tree 
after we compute the accuracy of the tree - acc 

we also will identify rows that were misclassified 


update the weights of according to the rows that 
we dibt snoky taje a vote 
we must take weighted vote accoding to how trustworthy each tree was 

acc 0.9 
acc 0.73 different tree
...

up to B trees ->> 


<a id='adaboost'></a>
## AdaBoost

---

AdaBoost is the original boosting algorithm. Predictions from AdaBoost follow this formula:


### $$ AdaBoost(X) = sign\left(\sum_{t=1}^T\alpha_t h_t(X)\right) $$

Where:

$AdaBoost(X)$ is the classification predictions for $y$ using predictor matrix $X$.

$T$ is the set of "weak learners.”

$\alpha_t$ is the contribution weight for weak learner $t$.

$h_t(X)$ is the prediction of weak learner $t$.

$y$ is binary **with values of negative one and one.*

The core principle of AdaBoost is to **fit a sequence of weak learners** (i.e., models that are only slightly better than random guessing, such as a single-split tree) **on repeatedly modified versions of the data**. After each fit, the importance weights on each observation need to be updated. 

The predictions are then combined through a weighted majority vote (or sum) to produce the final prediction. AdaBoost, like all boosting ensemble methods, focuses the next model's fit on the misclassifications/weaknesses of the prior models.

All training examples start with equal importance weighting. When we finish training a classifier, we update the importance weighting of the classifier itself, represented by alpha $\alpha$:

### $$ \alpha_t = \frac{1}{2}ln \left(\frac{1-\epsilon_t}{\epsilon_t}\right) \text{where } \epsilon_t < 1$$

Where $\epsilon_t$ is the misclassification rate for the current classifier:

### $$ \epsilon_t = \frac{\text{misclassifications}_t}{\text{observations}_t} $$

As iterations continue, **examples that are difficult to predict receive ever-increasing influence**. Each subsequent weak learner is thereby forced to concentrate on the examples that are missed by the previous ones in the sequence.

<a id='adaboost-viz'></a>
## AdaBoost Visualization

![boostvsbag](./images/adaboost-viz.png)

In [1]:
import numpy as np

In [2]:
def compute_alpha(acc):
    error = 1 - acc 
    return 0.5 * np.log((1-error)/error)

In [3]:
compute_alpha(0.6)

0.2027325540540821

more accurate more weight 

<a id='ex-weights'></a>
### Training Example Weights

AdaBoost sets up a weight vector on the observations, denoted as $D_t$ where $t$ is the current model iteration. $D_t$ is a probability distribution that determines how likely it is that a given observation will be selected as part of the training set for the current estimator.

The $\alpha$ weighting of the last-fit estimator is used in the equation for the weighting distribution. The update equation is:

### $$ D_{t+1}(i) = D_t(i) \cdot e^{-\alpha_t \cdot y_i \cdot h_t(x_i)} = D_t(i) \cdot (e^{\alpha_t})^{-y_i \cdot h_t(x_i)}.$$

- $D_t$ is the vector of observation indices, $D_t(i)$ is the ith datapoint in $D_t$,
- $x_i$ is the observation at the index, $y_i$ is the target, and
- $h_t$ is the previous model fit in the boosting chain.

> Note that $e^{\alpha_t}$ is always positive, and $-y_i h_t(x_i)$ is either -1 (if the sample is correctly classified) or +1 (if the sample is incorrectly classified).

Lastly, we'll divide the weights by the sum of weights to normalize them, ensuring that they sum to one and form a probability distribution:

### $$ D_{t+1}(i) = \frac{D_{t+1}(i)}{\sum_{i=1}^N D_{t+1}(i)}$$

-yht(x)  will give prediction of the Tth tree

what happens if yI and prediction is the same --> + * + become + but with the -yhx becomes -1  (hence will help to reduce the weights)

## Notes

In [4]:
# we compute D_t and D_t+1 
# these are both vectors of weights 
# that vector has length equivalent to the number 
# of rows in our dataset

# learning rate -> if higher learning rate, actual D_(t+1) that we use,
# will be closer to D(t+1)

#if the learning rate is lower 
# then the actual D(t+1) will be clsoer to D_t

<a id='gradient'></a>
## Gradient Boosting Models

---

Gradient boosting classifiers are a generalization of boosting to arbitrary, differentiable loss functions. The intuition behind this mechanism is to:

1. Fit a model $F$ to the data.
2. Look at the difference between our observed $y$ and our model $F$. (The $y_i - F(x_i)$ can be thought of as residuals!)
3. Fit a second model, $F_2$, to (roughly) the residuals $y_i - F(x_i)$.
4. Aggregate your model $F$ and $F_2$. While we won't get into the details now, we can interpret residuals as negative gradients. By doing this, we can apply our gradient descent algorithm to optimize our loss and generalize this to many loss functions.

GBRT is an accurate and effective off-the-shelf procedure that can be used for both regression and classification problems. Gradient tree boosting models are used in a variety of areas, including web search ranking and ecology.

[This presentation](http://www.ccs.neu.edu/home/vip/teach/MLcourse/4_boosting/slides/gradient_boosting.pdf) shows a visual example of AdaBoost at work and how AdaBoost laid the groundwork for gradient boosting classifiers.

**The advantages of GBRT are:**

- Natural handling of mixed data types (= heterogeneous features).
- Predictive power.
- Robustness to outliers in output space (via robust loss functions).

**The disadvantages of GBRT are:**
- Scalability: Due to the sequential nature of boosting, it can hardly be parallelized.
- Difficult hyperparameters to tune.


> _For more detailed explanations, see [here](https://www.quora.com/What-is-an-intuitive-explanation-of-Gradient-Boosting) and [here](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3885826/)._


<a id='gboost-viz'></a>
## Gradient Boosting Visualization
![boostvsbag](./images/gboost-viz.png)

<a id='the-difference-between-the-adaboost-and-gradient-boosting'></a>
## The Difference Between the AdaBoost and Gradient Boosting

---


- AdaBoost is about reweighting the preceding model's errors in subsequent iterations.
- Gradient boosting is about fitting subsequent models to the residuals of the last model.

## Playground

---

Let's practice boosting in `sklearn`.

## What's next?

---

There is no lab today!  You can choose your own adventure instead.  Below are some options for how you may want to spend your time.

- Practice boosting on a your own dataset.
- Take a deep-dive into how AdaBoost works by implementing it from scratch in the `adaboost-from-scratch.ipynb` notebook. (Solutions provided)
- `XGBoost` is a popular boosting package, seperate from `sklearn`.  (Many Kaggle competitions have been won with XGBoost.)  Try installing it and using it on a sample dataset (maybe the diabetes data included with this repo.) [http://xgboost.readthedocs.io/en/latest/](http://xgboost.readthedocs.io/en/latest/)
- Work on project 3
- Catch up on labs.

<a id='resources'></a>

## Additional Resources

---

- [Random Forest on Wikipedia](https://en.wikipedia.org/wiki/Random_forest)
- [Quora Question on Random Forests](https://www.quora.com/How-does-randomization-in-a-random-forest-work?redirected_qid=212859)
- [Scikit-Learn Ensemble Methods](http://scikit-learn.org/stable/modules/ensemble.html)
- [Scikit-Learn Random Forest Classifiers](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
- [Academic Introduction to Adaptive Boosting](http://rob.schapire.net/papers/explaining-adaboost.pdf)
- [Stack Exchange AdaBoost vs. Gradient Boosting](http://stats.stackexchange.com/questions/164233/intuitive-explanations-of-differences-between-gradient-boosting-trees-gbm-ad)
- [A Gentle Introduction to Gradient Boosting](http://www.ccs.neu.edu/home/vip/teach/MLcourse/4_boosting/slides/gradient_boosting.pdf)
- [Quora on Intuitive Explanations of AdaBoost](https://www.quora.com/What-is-AdaBoost)
- A Lighter [Math Introduction](http://www.ccs.neu.edu/home/vip/teach/MLcourse/4_boosting/slides/gradient_boosting.pdf) to AdaBoosting and Gradient Boosting
- A [Walk Through](https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/) on Tuning Gradient Boosting Models
